# Einleitung


# Imports

Zuerst müssen die benötigten Namespaces und Funktionen aus den verschiedenen Libraries importiert werden. 
Um alle Funktionen aus fastai zu importieren könnte man auch das verwenden:
```python
from fastai import *
from fastai.vision import *
```
Diese Methode ist jedoch eher unschön, da man den Überblick nicht hat, welche Namespaces man importiert hat. Falls man so mehrere Namespaces mit dem selbden Namen importiert, dann gewinnt der letzte. Als Beispiel: fastai hat einen Namespace mit dem Namen `Model`, welchen man hier nicht explizit gebraucht wird. Pytorch hat ebenfalls einen Namespace mit dem Namen `Model`, welcher gebraucht wird. Wenn man also `*` importiert kann es zu verwirrung führen welcher `Model`-Namespace jetzt verwendet wird oder von welcher Library dieser überhaupt kommt.

Darum ist es immer besser nur explizit Namespaces und Funktionen zu importieren

In [ ]:
from fastcore.foundation import L
from fastcore.xtras import Path # @patch'd properties to the Pathlib module

from fastai.callback.schedule import fit_one_cycle, lr_find 

from fastai.data.block import CategoryBlock, DataBlock
from fastai.data.transforms import get_image_files, RandomSplitter

from fastai.learner import Learner

from fastai.metrics import error_rate
from torchvision.models.resnet import resnet34
import torchvision.models as torch_models


from fastai.vision.all import (
    aug_transforms,
    ImageBlock,
    RegressionBlock,
    vision_learner,
    PILImage,
)

import json
import matplotlib.pyplot as plt

# Datei-Ablage

In diesem Abschnitt wird definiert, in welchem Ordner die Trainingsdaten gespeichert sind. Hier wird angenommen, das in dem Ordner immer pro Bild-Datei eine JSON-Datei mit demselben Namen befindet. Anschliessend wird überprüft, ob der Ordner existiert und das Resultat wird ausgegeben.

- `image_path`: Variablenname für den Dateipfad

In [ ]:
image_path = Path('data/results/<meine_resultate>')
image_path.exists()

# Daten aus einer JSON-Datei auslesen
In einem nächsten Schritt wird eine Funktion definiert, bei die nötigen Daten aus einer JSON-Datei ausliest und in einer Liste zurück gibt:

```python
def read_data_from_json(image):
```

- `image`: Ist der Posix Pfad zu dem Bild

In [ ]:
def read_data_from_json(image):
    with open(image_path/f'{image.stem}.json') as f:
        json_file = json.load(f)
        return[float(json_file['throttle']), float(json_file['steering'])]

# Testen

Nun ein kruzer Test der bereits erstellen Funktionalitäten. Dafür wird ein Bild geladen als PILImage. 
So kann man ganz einfach ein PILImage aus einem bestehenden Bild erstellen und anzeigen lassen:


In [ ]:
img = PILImage.create(image_path/'2022-10-18T14:15:52.011617.png')
img.show()

Als nächstes noch die `read_data_from_json` Funktion aufrufen

In [ ]:
read_data_from_json(Path(image_path/'2022-10-18T14:15:52.011617.png'))

# Datenblock

Jetzt ist alles so vorbereitet, dass man das Training mit der AI aufsetzten kann. Um damit zu beginnen muss zuerst ein `Datablock` erstellt werden. Ein Datablock ist einfach gesagt nichts anderes als ein Packet für eine genormte Schnittstelle. Das heisst, dass es sehr einfach ist anzugeben, was für Daten sind der Input, von wo kommen diese Daten, was ist der Ziel Output etc.

Da es sich hier um das Trainieren handelt werden sowohl der Erwartete Input, wie auch der dazugehörige Output benötigt.

Ein Datablock erstellt man relativ einfach:

In [ ]:
data_block = DataBlock(blocks=(ImageBlock, RegressionBlock(n_out=2)),
                    get_items=get_image_files,
                    splitter=RandomSplitter(),
                    get_y=read_data_from_json)

- `blocks`: Definiert mit welchen Daten das Model arbeiten wird. Meistens werden mind. zwei Blöcke definiert. Der erste Block spezifiziert die Input-Daten und der zweite Block die erwarteten Output-Daten sind. In diesem Beispiel werden Bilder als Input erwartet und als Output ein RegressionBlock mit zwei Ausgabewerten. Anhand dieser Regression wird versucht einem Bild Werte zuzuweisen. Der Grund wieso es zwei Outputs hat ist, dass es einmal für die Steuerung und einmal für die Geschwindigkeit ist.
- `get_items`: Von wo die Input-Daten gelesen werden. In diesem Fall wird die Methode `get_image_files` verwendet, welche Bilder aus einem Verzeichnis lädt.
- `splitter`: Für das Trainieren von Daten muss bestummen werden, welche Daten für das Training und welche für die anschliessende Validierung genutzt werden. Dies kann man zufällig machen mit einem `RandomSplitter`, aber es gibt auch Methoden, wo der Nutzer dies selber definieren kann z.B. einem `FileSplitter`
- `get_y`: Wie der Output definiert wird von den Input-Daten. Hier wird die Methode 'read_data_from_json' genutzt, beiwelcher definiert wurde, dass Anhand eines Bildes die Steuerung und das Tempo ausgelesen wird.
- `item_tfms`: Falls Bilder zu gross sind, kann diese Methode verwendet werden, um eine Transformation auf allen Bildern auszuführen, wie z.B. zuscheiden, verkleiner, hereinzoomen etc.

# Dataloaders

Mit der fertigstellung des DataBlocks ist nun definiert, wie unsere Daten Strukturiert, Kategorisiert und Bearbeitet werden müssen. Als nächstes folg die Erstellung eines `DataLoaders`.

In [ ]:
data_loaders = data_block.dataloaders(Path(image_path), bs=4)

Zuerst wird der Pfad angegeben, bei welcher der DataBlock die Bilder sucht. Die Variable `bs` steht für die Batch_Size.

Ein DataLoaders ist eine Iterator-Klasse, welche vom DataBlock aufgerufen wird um die Daten in Chunks zu laden anhand der definierten Batch-Size. Also der DataLoaders erzeugt mehrere `DataLoader` Einheiten. In einem DataLoader sind die Daten dann bereits vorbereitet und ready.

Man kann so ein Batch sich auch anzeigen lassen

In [ ]:
data_loaders.show_batch()

# Learner

Der nächste Schritt ist es eine `learner` zu erstellen. Der Learner fasst ein `Model`, mehrere `DataLoader` und eine `loss_function` zusammen in eine Methode. Die Dataloader wurden bereits erstellt, bzw. Die Iterator-Klasse `DataLoaders`.

Die erste Frage ist es, welche Art von `Learner` gebraucht wird. Hier wird ein `Vision Learner` genutzt. Der Vision Learner ist ein sogenannter CNN (Convolutional Neural Network) Learner. Ein CNN ist ein Netzwerk, dass für die Erkennung und Klassifizierung von Bildern genutzt wird. Anhand von übertragbaren lernen kann dieses Netzwerk für die Bildererkennung genutzt werden.

## Model
Eine Wichtige Entscheidung, die hier getroffen werden muss, ist die Entscheidung welches `Model` vewendet werden soll. Bei einem Modell handelt es sich um ein bereits trainiertes Modell mit vielen bis zu sehr vielen Daten.

Es gibt eine grosse Auswahl von verscheidenen Modellen. Die Auswahl ein richtiges Modell ist keine Einfache. Der Unterschied zwischen den Models ist, dass das Trainieren viel länger dauern kann, dafür aber je nachdem auch genauer ist. Jedoch ist das grösste Model auch nicht immer die beste Wahl, da ein grössere Modell auch für die Auswertung länger braucht.

Aus diesen Gründen ist die Wahl des Modells sehr wichtig und auch keine Einfache.

In [ ]:
learn = vision_learner(dls, resnet34)

# Learning Rate (Lern-Rate)

Bevor man anfängt das Model zu trainieren muss die `learning rate` gefunden werden. Dabei ist es wichtig, dass eine gute Lern-Rate gefunden wird:

- `Tiefe Lern-Rate`: Bei einer tiefen Lern-Rate dauert es länger bis das Modell einen optimalen Zustand erreicht. Das bedeutet es werden mehr Ressourcen und mehr Zeit gebraucht, als wenn man die Lern-Rate berechnet.
- `Hohe Lern-Rate`: Bei einer hohen Lern-Rate macht das Modell oft zu grosse Schritte, was dazu führen kann, dass der optimal Zustand überschossen wird und das Modell im gegenzug and Qualität verliert.

In [ ]:
learn.lr_find()

In [ ]:
lr = 3.630780702224001e-05

Nun ist es endlich an der Zeit das Modell zu trainieren

In [ ]:
learn.fine_tune(25, lr)

Beim `fine_tune` wird sowohl der `train_loss`, wie auch der `valid_loss` angezeigt:

- `train_loss`: Der Trainings-Verlust zeigt an, wie passend das Modell zu den Trainingsdaten ist. Je kleiner dieser Wert ist desto besser.
- `valid_loss`: Der Validierungs-Verlust zeigt an, wie passend das Modell zu den neuen Daten ist. Je kleiner dieser Wert ist desto besser.

# Fertig mit dem Training

Wenn man fertig mit dem Trainieren ist, dann kann man sich auch das Resultat anzeigen lassen

In [ ]:
learn.show_results(ds_idx=1)

Oder auch mal ein Testbild vorhersagen lassen

In [ ]:
learn.predict('<pfad>/<zum>/<Bild>')

Oder sogar für mehrere Bilder auf einmal

In [ ]:
files = get_image_files('<pfad-zu-den-Bildern>')
test_dl = learn.dls.test_dl(files)
preds = learn.get_preds(dl=test_dl)
preds

# Exportieren der Daten

Daten können ganz einfach in ein `Pickle`-File exportiert werden. So können sie später mit der `load_learner`-Funktion einfach wieder geladen werden. 

**Achtung** Damit der Learner wieder importiert und gebraucht werden kann müssen auch die Funktionen, die im DataBlock benutzt wurden am neuen Ort erstellt werden. In diesem Fall ist das die Funktion `read_data_from_json`.

In [ ]:
learn.export(Path('<Pfad>/<Modellname>.pkl'))

# Importieren

Wenn der Learner importiert wurde, kann dieser wie vorhin weiter verwendet werden.

In diesem Fall wird der Learner dann auf dem Fahrzeug importiert und genutzt, damit dieses so gut wie möglich selbst fahren kann.

In [ ]:
learn = load_learner('<Pfad>/<Modellname>.pkl')